In [3]:
from models import create_model
from util.visualizer import Visualizer
from dataset.datahandler import get_data_loader
import numpy as np
import torch
from util.lidar import LiDAR
import yaml
from train import make_class_from_dict,M_parser
import random

In [4]:
class DotDict:
    def __init__(self, **entries):
        self.__dict__.update(entries)

# Convert dictionary to a DotDict instance
cl_args={
    'cfg_path':'configs/cut.yaml',
    'data_dir': "Semantic-CARLA/CARLA_Lidar/projected",
    'data_dir_B':"/media/pierre/Main/SemanticKitti/dataset",
    'batch_size':1,
    'load':None,
    'test':False,
    'ref_dataset_name':None,
    'norm_label': True,
    'on_real': False
}
cl_args = DotDict(**cl_args)
opt = M_parser(cl_args.cfg_path, cl_args.data_dir,cl_args.data_dir_B, cl_args.load, cl_args.test, cl_args.batch_size)
if cl_args.on_real:
    opt= cl_args.ref_dataset_name
opt.model.norm_label = cl_args.norm_label
# opt.training.n_epochs = 1
torch.manual_seed(opt.training.seed)
np.random.seed(opt.training.seed)
random.seed(opt.training.seed)
is_two_dataset = False
if hasattr(opt.dataset, 'dataset_B'):
    is_two_dataset = True
device = torch.device('cuda:0') 


In [5]:
ds_cfg = make_class_from_dict(yaml.safe_load(open(f'configs/dataset_cfg/{opt.dataset.dataset_A.name}_cfg.yml', 'r')))
lidar_A = LiDAR(
cfg=ds_cfg,
height=opt.dataset.dataset_A.img_prop.height,
width=opt.dataset.dataset_A.img_prop.width).to(device)

ds_cfg_B = make_class_from_dict(yaml.safe_load(open(f'configs/dataset_cfg/{opt.dataset.dataset_B.name}_cfg.yml', 'r')))
lidar_B = LiDAR(
cfg=ds_cfg_B,
height=opt.dataset.dataset_B.img_prop.height,
width=opt.dataset.dataset_B.img_prop.width,
).to(device)

In [6]:
visualizer = Visualizer(opt)   # create a visualizer that display/save images and plots
g_steps = 0
min_fid = 10000
# if cl_args.ref_dataset_name == 'kitti':
#     ignore_label = [0, 2, 3, 4, 5, 7, 8, 10, 12, 16]
# elif cl_args.ref_dataset_name == 'semanticPOSS':
#     ignore_label = [0, 3, 9]
# is_ref_semposs = cl_args.ref_dataset_name == 'semanticPOSS'
is_ref_semposs=False
train_dl, train_dataset = get_data_loader(opt, 'train', opt.training.batch_size,is_ref_semposs=is_ref_semposs)
val_dl, val_dataset = get_data_loader(opt, 'val' , opt.training.batch_size, shuffle=False, is_ref_semposs=is_ref_semposs)  
# test_dl, test_dataset = get_data_loader(opt, 'test', opt.training.batch_size, dataset_name=cl_args.ref_dataset_name, two_dataset_enabled=False, is_ref_semposs=is_ref_semposs)
   

Subsets  [ 0  1  2  3  4  5  6  7  9 10]
taille data 33992
Subsets  [ 0  1  2  3  4  5  6  7  9 10]
taille data 19130
Subsets  [8]
taille data 0
Subsets  [8]
taille data 4071


In [7]:
model = create_model(opt, lidar_A, lidar_B)
#First FOWARD
train_dl_iter=iter(train_dl)
data=next(train_dl_iter)
model.set_input(data)        
#SET pathcLoss Network
model.data_dependent_initialize(data) #Set up the PatchNetwork
# model.optimize_parameters()  
model.setup(opt.training)

<util.make_class_from_dict.<locals>.dict_class object at 0x724c40e75180>
initialize network with xavier
initialize network with xavier
initialize network with xavier
initialize network with xavier
model [CUTModel] was created
initialize network with xavier


/home/pierre/Documents/git_folders/CoLiGen/models/networks.py:611: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  patch_id = torch.tensor(patch_id, dtype=torch.long, device=feat.device)


---------- Networks initialized -------------
ResnetGenerator(
  (gumbel): GumbelSigmoid(hard=True, eps=1e-10)
  (model): Sequential(
    (0): ReflectionPad2d((3, 3, 3, 3))
    (1): Conv2d(2, 64, kernel_size=(7, 7), stride=(1, 1))
    (2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (3): ReLU(inplace=True)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (6): ReLU(inplace=True)
    (7): Downsample(
      (pad): ReflectionPad2d((1, 1, 1, 1))
    )
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (10): ReLU(inplace=True)
    (11): Downsample(
      (pad): ReflectionPad2d((1, 1, 1, 1))
    )
    (12): ResnetBlock(
      (conv_block): Sequential(
        (0): ReflectionPad2d((1, 1, 1, 1))
        

In [8]:
from tqdm import tqdm
num_epoch=opt.training.epochs
num_epoch=1
train_dl_iter = iter(train_dl)
print(len(train_dl_iter))
for epoch in tqdm(range(0, len(train_dl_iter))):  # inner loop within one epoch
    e_steps = 0      
    model.train(True)
    data = next(train_dl_iter)
    model.set_input(data)         # unpack data from dataset and apply preprocessing
    model.optimize_parameters()   # calculate loss functions, get gradients, update network weights
model.save_networks(opt.training.epochs)


33992


100%|██████████| 33992/33992 [3:28:22<00:00,  2.72it/s]  


In [9]:
model.save_networks(1)

In [11]:
import  matplotlib.pyplot as plt
from models import create_model
from dataset.datahandler import get_data_loader
import numpy as np
import torch
from util.lidar import LiDAR
import yaml
from train import make_class_from_dict
from collections import defaultdict
from tqdm import tqdm


class M_parser():
    def __init__(self, cfg_path, data_dir):
        opt_dict = yaml.safe_load(open(cfg_path, 'r'))
        dict_class = make_class_from_dict(opt_dict)
        members = [attr for attr in dir(dict_class) if not callable(getattr(dict_class, attr)) and not attr.startswith("__")]
        for m in members:
            setattr(self, m, getattr(dict_class, m))
        if data_dir != '':
            self.dataset.dataset_A.data_dir = data_dir
        self.isTrain = False

cfg_test="configs/cut_test.yaml"
data_dir="Semantic-CARLA/CARLA_Lidar/projected"
device="cuda"
is_ref_semposs=False

ds_cfg = make_class_from_dict(yaml.safe_load(open(f'configs/dataset_cfg/carla_cfg.yml', 'r')))
opt = M_parser(cfg_test,data_dir)
opt.model.norm_label = True
torch.manual_seed(opt.seed)
np.random.seed(opt.seed)
lidar_A = LiDAR(
    cfg=ds_cfg,
    height=opt.dataset.dataset_A.img_prop.height,
    width=opt.dataset.dataset_A.img_prop.width).to(device)
print(lidar_A)


test_dl, test_dataset = get_data_loader(cfg=opt, split='test',batch_size=1 ,shuffle=False,is_ref_semposs=is_ref_semposs,two_dataset_enabled=False)
model = create_model(opt,lidar_A)# create a model given opt.model and other options

opt.epoch=100
opt.verbose=False

model.setup(opt)              # regular setup: load and print networks; create schedulers

test_losses = defaultdict(list)
test_image_results = defaultdict(list)
model.train(False)
n_pics = 0
generated_remission = []

number_tests=33
print(len(test_dl))
for i in tqdm(range(number_tests)):
    test_dl_iter = iter(test_dl)
    n_test_batch = 50 
    for i in range(n_test_batch):
        data = next(test_dl_iter)

        model.set_input(data)
        with torch.no_grad():
            model.forward()
print(data['path'])
"00/000033.npy"

LiDAR()
Subsets  [0]
taille data 6999
initialize network with xavier
initialize network with xavier
initialize network with xavier
model [CUTModel] was created
loading the model from coligen/e_100.pth
---------- Networks initialized -------------
[Network G] Total number of parameters : 11.372 M
[Network C] Total number of parameters : 5.102 M
-----------------------------------------------
6999


100%|██████████| 33/33 [01:32<00:00,  2.82s/it]

['Semantic-CARLA/CARLA_Lidar/projected/sequences/00/velodyne/000049.npy']


'00/000033.npy'

In [8]:
model.load_networks("latest")

loading the model from coligen/latest.pth


In [9]:
import matplotlib.pyplot as plt
model.real_B_inv.shape
real_B_inv=model.real_B_inv.detach().cpu().numpy()[0][0]
plt.imsave("real_B_inv.png",real_B_inv)

In [10]:
model.real_B_mask.shape
real_B_mask=model.real_B_mask.detach().cpu().numpy()[0][0]
plt.imsave("real_B_mask.png",real_B_mask)

In [11]:
model.real_inv.shape
real_inv=model.real_inv.detach().cpu().numpy()[0][0]
plt.imsave("real_inv.png",real_inv)

In [12]:
model.synth_inv.shape
synth_inv=model.synth_inv.detach().cpu().numpy()[0][0]
plt.imsave("synth_inv.png",synth_inv)

In [13]:
model.synth_reflectance.shape
synth_reflectance=model.synth_reflectance.detach().cpu().numpy()[0][0]
plt.imsave("synth_reflectance.png",synth_reflectance)

In [28]:
attrs = vars(model)
print(attrs.keys())
print(', '.join("%s: %s" % item for item in attrs.items()))

dict_keys(['opt', 'isTrain', 'gpu_ids', 'save_dir', 'device', 'eval_metrics', 'loss_names', 'extra_val_loss_names', 'model_names', 'visual_names', 'optimizers', 'image_paths', 'crterionSSIM', 'metric', 'lidar_A', 'lidar_B', 'nce_layers', 'netC', 'netG', 'netF', 'netF_feat', 'netD', 'criterionGAN', 'criterionNCE', 'criterionIdt', 'optimizer_G', 'optimizer_D', 'schedulers', 'real_inv', 'real_mask', 'real_depth', 'real_points', 'real_lwo', 'real_reflectance', 'real_rgb', 'real_label', 'real_path', 'real_A', 'cond_A', 'data_A', 'real_B_inv', 'real_B_mask', 'real_B_depth', 'real_B_points', 'real_B_lwo', 'real_B_reflectance', 'real_B_rgb', 'real_B_label', 'real_B_path', 'real_B', 'cond_B', 'real_B_mod_A', 'real_A_mod_B', 'data_B', 'real', 'cond', 'cond_feat', 'fake', 'fake_B', 'synth_inv', 'synth_reflectance', 'synth_mask', 'loss_D_fake', 'pred_real', 'loss_D_real', 'loss_D', 'loss_G_GAN', 'loss_NCE_pix', 'loss_NCE_feat', 'loss_NCE_bd', 'loss_G', 'optimizer_F'])
opt: <train.M_parser object a

In [14]:
model.synth_mask.shape
synth_mask=model.synth_mask.detach().cpu().numpy()[0][0]
plt.imsave("synth_mask.png",synth_mask)

In [16]:

model.pred_real.shape
pred_real=model.pred_real.detach().cpu().numpy()[0][0]
plt.imsave("pred_real.png",pred_real)

In [17]:
model.fake.shape
fake=model.pred_real.detach().cpu().numpy()[0][0]
plt.imsave("fake.png",fake)

In [18]:
model.fake_B.shape
fake_B=model.fake_B.detach().cpu().numpy()[0][0]
plt.imsave("fake_B.png",fake_B)

In [22]:

model.real_rgb.shape
real_rgb=model.real_rgb.detach().cpu().numpy()[0][0]
plt.imsave("real_rgb.png",real_rgb)

In [19]:
file_path = f"/home/pierre/Documents/git_folders/CoLiGen/Semantic-CARLA/CARLA_Lidar/projected/sequences/06/velodyne/000768.npy"  # Replace with your .bin file path
# Semantic-CARLA/CARLA_Lidar/projected/sequences/06/velodyne/000768.npy
points = np.load(file_path).astype(np.float64)
depth = np.sqrt(np.power(points[:,:,0],2) + np.power(points[:,:,1],2) + np.power(points[:,:,2],2))
plt.imsave("depth.png",depth)
print(depth)


[[19.25949936 19.29892235 19.25338918 ... 19.08344456 19.06185161
  19.22045447]
 [19.24910573 19.28849269 19.24231221 ... 19.07280193 19.05121916
  19.21007357]
 [19.23989404 19.27923048 19.23241948 ... 19.0632967  19.04177815
  19.20089538]
 ...
 [ 0.          0.6195506   0.61958225 ...  0.61961891  0.61960008
   0.61950589]
 [ 0.          0.61152099  0.6115525  ...  0.61160679  0.61158807
   0.61149451]
 [ 0.60372179  0.5962139   0.59624495 ...  0.59628092  0.59624506
   0.5961875 ]]
